In [1]:
import os
import shutil
import cv2
import json
import numpy as np

# Funktion zum Extrahieren der Polygonpunkte und Bildabmessungen aus JSON
def extract_data_from_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    img_width = data['imageWidth']
    img_height = data['imageHeight']

    all_polygons = []
    for shape in data['shapes']:
        points = shape['points']
        all_polygons.append(points)

    return all_polygons, img_height, img_width

# Funktion zur Erstellung der weißen Polygon-Masken
def generate_white_polygons(json_file):
    polygons, img_height, img_width = extract_data_from_json(json_file)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)

    for polygon in polygons:
        points = np.array(polygon, dtype=np.int32)
        cv2.fillPoly(image, [points], color=(255, 255, 255))

    return image, img_height, img_width

# Funktion zum Kopieren der RGB-Bilder und Erstellen der Ground-Truth-Masken
def create_rgb_and_gt_folders(source_dirs, rgb_output_dir, gt_output_dir):
    # Sicherstellen, dass die Ausgabeverzeichnisse existieren
    if not os.path.exists(rgb_output_dir):
        os.makedirs(rgb_output_dir)
    if not os.path.exists(gt_output_dir):
        os.makedirs(gt_output_dir)

    # Listen für fehlende Dateien und Fehler beim Maskieren
    missing_pairs = []

    # Durchlaufen jedes Quellverzeichnisses
    for source_dir in source_dirs:
        annotated_dir = os.path.join(source_dir, f"{os.path.basename(source_dir)}_annotated")
        
        if os.path.exists(annotated_dir):  # Prüfen, ob das annotierte Verzeichnis existiert
            for root, dirs, files in os.walk(annotated_dir):
                for file in files:
                    if file.endswith(".png"):
                        # Konstruiere die Pfade für die RGB-Datei
                        rgb_source_path = os.path.join(root, file)
                        json_file = os.path.splitext(rgb_source_path)[0] + ".json"

                        # Nur wenn es sowohl eine .png- als auch eine .json-Datei gibt
                        if os.path.exists(json_file):
                            # Kopiere das RGB-Bild ins test_rgb-Verzeichnis
                            rgb_dest_path = os.path.join(rgb_output_dir, file)
                            shutil.copyfile(rgb_source_path, rgb_dest_path)

                            try:
                                # Generiere die Ground-Truth-Maske
                                mask, img_height, img_width = generate_white_polygons(json_file)

                                # Speichere die Maske mit demselben Namen wie die RGB-Datei im test_gt-Verzeichnis
                                gt_dest_path = os.path.join(gt_output_dir, file)
                                cv2.imwrite(gt_dest_path, mask)
                            except Exception as e:
                                print(f"Fehler beim Generieren der Maske für {file}: {e}")
                        else:
                            # Protokolliere fehlende Paare (falls eine .png ohne .json existiert)
                            missing_pairs.append(file)

    # Protokolliere die Dateien, die keine entsprechenden JSON-Dateien haben
    if missing_pairs:
        print("Die folgenden PNG-Dateien haben keine entsprechenden JSON-Dateien und wurden daher nicht verarbeitet:")
        for missing in missing_pairs:
            print(missing)

# Liste der Quellverzeichnisse
source_dirs = [
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling1',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling2',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling3',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling4',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling5',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling6',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling7',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling8',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling9',
    '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling10'
]

# Ausgabeverzeichnisse
rgb_output_dir = '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/test_rgb/'
gt_output_dir = '/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/test_gt/'

# Funktion zum Erstellen von RGB- und Ground-Truth-Ordnern ausführen
create_rgb_and_gt_folders(source_dirs, rgb_output_dir, gt_output_dir)

print("RGB-Bilder und Ground-Truth-Masken wurden erfolgreich erstellt.")


Die folgenden PNG-Dateien haben keine entsprechenden JSON-Dateien und wurden daher nicht verarbeitet:
soiling4_55.png
soiling4_58.png
soiling4_41.png
soiling4_29.png
soiling4_68.png
soiling4_66.png
soiling4_60.png
soiling4_56.png
soiling4_57.png
soiling4_64.png
soiling4_37.png
soiling4_71.png
soiling4_69.png
soiling4_70.png
soiling4_65.png
soiling4_45.png
soiling4_39.png
soiling4_35.png
soiling4_67.png
soiling4_47.png
soiling4_31.png
soiling4_63.png
soiling4_62.png
soiling4_54.png
soiling4_61.png
RGB-Bilder und Ground-Truth-Masken wurden erfolgreich erstellt.
